In [ ]:
# install the following dependencies (pip install)
# datasets
# seaborn
# scikit-learn
# kagglehub
# opencv-python

Download the IAM (human handwriting dataset) from kaggle. <br>
Download the Deepwriting (ai handwriting dataset) from https://ait.ethz.ch/deepwriting choose the dataset, then unzip the file and just use the .npz file <br>

OR DOWNLOAD AND UNZIP FROM THE GOOGLE DRIVE

In [ ]:
# download the IAM (human handwriting dataset) SKIP IF ALREADY HAVE THE DATASET
import kagglehub

# Download latest version
path = kagglehub.dataset_download("naderabdalghani/iam-handwritten-forms-dataset")

print("Path to dataset files:", path)

Path to dataset files: C:\Users\Carine\.cache\kagglehub\datasets\naderabdalghani\iam-handwritten-forms-dataset\versions\1


In [ ]:
# START
# Import required libraries
import os
import cv2
import numpy as np
import time
import pandas as pd

from sklearn.preprocessing import StandardScaler
from sklearn.utils import shuffle

In [ ]:
# Step 1: Process Both DeepWriting Training & Validation Datasets

# Paths to DeepWriting datasets
deepwriting_train_path = r"C:\Users\Carine\Desktop\deepwriting_training.npz" # CHANGE TO YOUR PATH
deepwriting_val_path = r"C:\Users\Carine\Desktop\deepwriting_validation.npz" # CHANGE TO YOUR PATH

# Output folders for processed images
train_output_path = r"C:\Users\Carine\Desktop\deepwriting_train" # CHANGE TO YOUR PATH
val_output_path = r"C:\Users\Carine\Desktop\deepwriting_val" # CHANGE TO YOUR PATH
os.makedirs(train_output_path, exist_ok=True)
os.makedirs(val_output_path, exist_ok=True)

# Function to load, convert strokes to images, and save them
def process_deepwriting(npz_path, output_folder):
    data = np.load(npz_path, allow_pickle=True)
    strokes = data["strokes"]  # Extract stroke sequences

    def strokes_to_image(stroke_data, img_size=128):
        x = np.cumsum(stroke_data[:, 0])
        y = np.cumsum(stroke_data[:, 1])

        x = ((x - x.min()) / (x.max() - x.min()) * (img_size - 1)).astype(np.int32)
        y = ((y - y.min()) / (y.max() - y.min()) * (img_size - 1)).astype(np.int32)

        img = np.ones((img_size, img_size), dtype=np.uint8) * 255  # White background

        for i in range(len(x) - 1):
            cv2.line(img, (x[i], y[i]), (x[i+1], y[i+1]), 0, thickness=2)

        return img

    for idx, stroke_data in enumerate(strokes):
        img = strokes_to_image(stroke_data)
        cv2.imwrite(os.path.join(output_folder, f"deepwriting_{idx}.png"), img)

    print(f"Processed {len(strokes)} images and saved to {output_folder}")

# Process training and validation datasets
process_deepwriting(deepwriting_train_path, train_output_path)
process_deepwriting(deepwriting_val_path, val_output_path)


Processed 34577 images and saved to C:\Users\Carine\Desktop\deepwriting_train
Processed 705 images and saved to C:\Users\Carine\Desktop\deepwriting_val


In [ ]:
# Step 2: Process IAM (Human Handwriting) Dataset

# **Updated IAM dataset path**
iam_dataset_path = r"C:\Users\Carine\.cache\kagglehub\datasets\naderabdalghani\iam-handwritten-forms-dataset\versions\1\data" # CHANGE TO YOUR PATH

# Function to load images recursively from all subfolders
def load_images_from_subfolders(parent_folder, label):
    images = []
    labels = []

    # **Loop through each subfolder**
    for subfolder in os.listdir(parent_folder):
        subfolder_path = os.path.join(parent_folder, subfolder)
        
        # Ensure it's a folder
        if not os.path.isdir(subfolder_path):
            continue  

        # **Process all images inside the subfolder**
        for file in os.listdir(subfolder_path):
            if file.endswith(".png") or file.endswith(".jpg"):  # Process only images
                img_path = os.path.join(subfolder_path, file)
                img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)  # Convert to grayscale
                
                if img is None:
                    print(f"Warning: Could not read image {file} in {subfolder}")
                    continue
                
                img = cv2.resize(img, (128, 128)) / 255.0  # Resize and normalize
                images.append(img)
                labels.append(label)

    return np.array(images), np.array(labels)

# **Load IAM dataset again (including subfolders)**
iam_images, iam_labels = load_images_from_subfolders(iam_dataset_path, label=0)

# **Check if images are loaded**
print(f"IAM Dataset Loaded: {iam_images.shape}, Labels: {iam_labels.shape}")


IAM Dataset Loaded: (1539, 128, 128), Labels: (1539,)


In [ ]:
# Step 3: Merge IAM & DeepWriting Datasets

# **Ensure IAM images have the correct shape**
iam_images = iam_images.reshape(-1, 128, 128)

# **Load DeepWriting Images**
deepwriting_train_path = r"C:\Users\Carine\Desktop\deepwriting_train" # CHANGE TO YOUR PATH
deepwriting_val_path = r"C:\Users\Carine\Desktop\deepwriting_val" # CHANGE TO YOUR PATH

# Function to load images from folder
def load_images_from_folder(folder_path, label):
    images, labels = [], []
    for file in os.listdir(folder_path):
        if file.endswith(".png") or file.endswith(".jpg"):
            img_path = os.path.join(folder_path, file)
            img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
            img = img / 255.0  # Normalize
            images.append(img)
            labels.append(label)
    return np.array(images), np.array(labels)

deepwriting_train_images, deepwriting_train_labels = load_images_from_folder(deepwriting_train_path, label=1)
deepwriting_val_images, deepwriting_val_labels = load_images_from_folder(deepwriting_val_path, label=1)

# **Check DeepWriting Shapes**
print(f"DeepWriting Train Shape: {deepwriting_train_images.shape}, Labels: {deepwriting_train_labels.shape}")
print(f"DeepWriting Validation Shape: {deepwriting_val_images.shape}, Labels: {deepwriting_val_labels.shape}")

# **Split IAM dataset into 80% train and 20% validation**
split_idx = int(0.8 * len(iam_images))
iam_train_images, iam_train_labels = iam_images[:split_idx], iam_labels[:split_idx]
iam_val_images, iam_val_labels = iam_images[split_idx:], iam_labels[split_idx:]

# **Merge AI (DeepWriting) & Human (IAM) datasets**
X_train = np.concatenate((deepwriting_train_images, iam_train_images), axis=0)
y_train = np.concatenate((deepwriting_train_labels, iam_train_labels), axis=0)
X_val = np.concatenate((deepwriting_val_images, iam_val_images), axis=0)
y_val = np.concatenate((deepwriting_val_labels, iam_val_labels), axis=0)

# **Shuffle dataset**
X_train, y_train = shuffle(X_train, y_train, random_state=42)
X_val, y_val = shuffle(X_val, y_val, random_state=42)

# **Save the final dataset**
dataset_path = r"C:\Users\Carine\Desktop\preprocessed_handwriting_dataset.npz" # CHANGE TO YOUR PATH
np.savez(dataset_path, X_train=X_train, y_train=y_train, X_val=X_val, y_val=y_val)

print(f"Final dataset saved with training shape {X_train.shape}, validation shape {X_val.shape}")

DeepWriting Train Shape: (34577, 128, 128), Labels: (34577,)
DeepWriting Validation Shape: (705, 128, 128), Labels: (705,)
Final dataset saved with training shape (35808, 128, 128), validation shape (1013, 128, 128)
